# Stock Analysis project
## Data gathering
The data was gathered from the Bovespa website in the historical quotes [link](http://www.bmfbovespa.com.br/pt_br/servicos/market-data/historico/mercado-a-vista/cotacoes-historicas/). The instructions to interpret the data are in the same website.

In [ ]:
%matplotlib inline

# All imports
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# Constants
RANDOM_SEED = 42
DAYS_AHEAD = 5 - 1  #DAYS_AHEAD = 0 is looking 1 day ahead, trying to know today with the data from the past

# Data gathering
stocks = ['ITSA4 ']
paths = ['stock_price\\inputs\\COTAHIST_A2006.TXT','stock_price\\inputs\\COTAHIST_A2007.TXT',
        'stock_price\\inputs\\COTAHIST_A2008.TXT','stock_price\\inputs\\COTAHIST_A2009.TXT']#,
        #'stock_price\\inputs\\COTAHIST_A2010.TXT','stock_price\\inputs\\COTAHIST_A2011.TXT',
        #'stock_price\\inputs\\COTAHIST_A2012.TXT','stock_price\\inputs\\COTAHIST_A2013.TXT',
        #'stock_price\\inputs\\COTAHIST_A2014.TXT','stock_price\\inputs\\COTAHIST_A2015.TXT']
df = pd.DataFrame()
for stock in stocks:
    for path in paths:
        file = open(path,'r')
        for line in file:
            if (line[12:18] == stock):
                df = df.append({'year': int(line[2:6]),'month': int(line[6:8]),'day': int(line[8:10]),
                                'open': int(line[56:69])/100.,'high': int(line[69:82])/100.,
                                'low': int(line[82:95])/100.,'close': int(line[108:121])/100.,
                                'volume': int(line[152:170]), 'stock': stocks.index(stock)},ignore_index=True)
        file.close
df.describe()

## Technical analysis
The main data has already been gathered. Now it is time to use what is called technical analysis to improve the data we already have.

The technical analysis consideres that the price movement of a stock reflects every thing that you need to know about the stock. The following image shows the analysis of a technical analist:

![Analysis of a technical analist](http://www.liberatedstocktrader.com/wp-content/uploads/stock-market-analysis-sept-20-2010.jpg)

To perform this analysis there is some indicators (calculated from the prizes) that are used for the behaviour prediction. We will use [Moving Averages](http://www.investopedia.com/terms/m/movingaverage.asp) for 5, 8, 21 and 50 periods.

It is important to notice that we will only have the data for previous periods and not from our period. So to start we will make every date know about its 50 previous closing prices

In [ ]:
# Knowing the past
for i in range(1,50 + 1):
    df['close'+str(i)] = -1
    for j in range(50,len(df)):
        df.loc[j,'close'+str(i)] = df.loc[j-i,'close']
for i in range(1,50 + 1):
    df['low'+str(i)] = -1
    for j in range(50,len(df)):
        df.loc[j,'low'+str(i)] = df.loc[j-i,'low']
for i in range(1,50 + 1):
    df['high'+str(i)] = -1
    for j in range(50,len(df)):
        df.loc[j,'high'+str(i)] = df.loc[j-i,'high']
for i in range(1,50 + 1):
    df['open'+str(i)] = -1
    for j in range(50,len(df)):
        df.loc[j,'open'+str(i)] = df.loc[j-i,'open']

# Verify if the code worked
plt.scatter(df.index.tolist(),df['close'].values,c='c')
plt.scatter(df.index.tolist(),df['close50'].values)

In [ ]:
# Create the moving averages
for i in (5,8,21,50):
    df['MA'+str(i)] = -1
    for j in range(50,len(df)):
        sum = 0
        for k in range(1,i+1):
            sum += df.loc[j,'close'+str(k)]
        df.loc[j,'MA'+str(i)] = sum/i

# Verify if the code worked
plt.scatter(df.index.tolist(),df['close'].values,c='c')
plt.plot(df.index.tolist(),df['MA50'].values,'-',c='k')

We also want to know which day of the week it is. There is some studies that argue that a stock has a higher probability of moving up depending of the day of the week.

In [ ]:
df['week_day']=-1
for i in range(len(df)):
    df.loc[i,'week_day'] = datetime.datetime(int(df.loc[i,'year']), int(df.loc[i,'month']), int(df.loc[i,'day'])).weekday()
# Just for information Monday to Sunday is 0 to 6

The volume that we have are also from the last days. In the case of the volume we will gather only the volume for the last 5 days.

In [ ]:
for i in range(1,5 + 1):
    df['volume'+str(i)] = -1
    for j in range(5,len(df)):
        df.loc[j,'volume'+str(i)] = df.loc[j-i,'volume']

# Verify if the code worked
plt.scatter(df.index.tolist(),df['volume'].values,c='c')
plt.scatter(df.index.tolist(),df['volume5'].values)

## Preparing the data to use
Since the price of the stock varies alot, the important aspect here is the movement relative to the last price. So every price show be divided by the last price (close1) so we have the relative price of the movements until here.
We will do the same for the volumes (volume1).

In [ ]:
df.loc[:,'close'] = df.loc[:,'close']/df.loc[:,'close1']
df.loc[:,'low'] = df.loc[:,'low']/df.loc[:,'close1']
df.loc[:,'high'] = df.loc[:,'high']/df.loc[:,'close1']
df.loc[:,'open'] = df.loc[:,'open']/df.loc[:,'close1']

for i in range(2,50+1):
    df.loc[:,'close'+str(i)] = df.loc[:,'close'+str(i)]/df.loc[:,'close1']
    df.loc[:,'low'+str(i)] = df.loc[:,'low'+str(i)]/df.loc[:,'close1']
    df.loc[:,'high'+str(i)] = df.loc[:,'high'+str(i)]/df.loc[:,'close1']
    df.loc[:,'open'+str(i)] = df.loc[:,'open'+str(i)]/df.loc[:,'close1']
    
for i in (5,8,21,50):
    df.loc[:, 'MA'+str(i)] = df.loc[:, 'MA'+str(i)]/df.loc[:,'close1']

df.loc[:,'open1'] = df.loc[:,'open1']/df.loc[:,'close1']
df.loc[:,'high1'] = df.loc[:,'high1']/df.loc[:,'close1']
df.loc[:,'low1'] = df.loc[:,'low1']/df.loc[:,'close1']
df.loc[:,'close1'] = df.loc[:,'close1']/df.loc[:,'close1']

We will do the same for the volumes (volume1).

In [ ]:
df.loc[:,'volume'] = df.loc[:,'volume']/df.loc[:,'volume1']

for i in range(2,5+1):
    df.loc[:,'volume'+str(i)] = df.loc[:,'volume'+str(i)]/df.loc[:,'volume1']
    
df.loc[:,'volume1'] = df.loc[:,'volume1']/df.loc[:,'volume1']

## Clean the data
Since the data is now read, we need to erase the 50 first days. They do not have all the indicators that we need for the analysis and it can bring some problems.

In [ ]:
df = df.loc[50:].reset_index()

df.describe()

## Preparing the target
Until now we have not prepared the target, the target will be a data frame that gives 1 when price close higher than price open, otherwise it returns 1.

In [ ]:
y_all = []

for i in range(len(df)-DAYS_AHEAD):
    is_higher = (1 if (df.loc[i+DAYS_AHEAD,'close']/df.loc[i,'open'] > 1.) else 0)
    y_all.extend([is_higher])
    
df.loc[:len(df)-DAYS_AHEAD-1,'up'] = y_all
    
df = df.dropna()

plt.hist(y_all,bins=3)
plt.show()

len(y_all)

We can see that it is almost a 50% chance for the day to close higher or not.

In [ ]:
df.describe()

## Prepare data
Now that we have the target we can clean our data to start the work.

In [ ]:
X_all = df.drop(['index','close','low','high','open','volume','up','day'],1)

X_all.describe()

# Data analysis
Let's use the data to plot some small studies about the behaviour of the y_all.
## Day of the week

In [ ]:
plt.subplot(321)
plt.hist(df[df['week_day']==0]['up'],bins=3)
plt.title('Monday')
plt.subplot(322)
plt.hist(df[df['week_day']==1]['up'],bins=3)
plt.title('Tuesday')
plt.subplot(323)
plt.hist(df[df['week_day']==2]['up'],bins=3)
plt.title('Wednesday')
plt.subplot(324)
plt.hist(df[df['week_day']==3]['up'],bins=3)
plt.title('Thrusday')
plt.subplot(325)
plt.hist(df[df['week_day']==4]['up'],bins=3)
plt.title('Friday')
plt.show()


## Month of the year

In [ ]:
plt.subplot(431)
plt.hist(df[df['month']==1]['up'],bins=3)
plt.title('Jan')
plt.subplot(432)
plt.hist(df[df['month']==2]['up'],bins=3)
plt.title('Feb')
plt.subplot(433)
plt.hist(df[df['month']==3]['up'],bins=3)
plt.title('Mar')
plt.subplot(434)
plt.hist(df[df['month']==4]['up'],bins=3)
plt.title('Apr')
plt.subplot(435)
plt.hist(df[df['month']==5]['up'],bins=3)
plt.title('May')
plt.subplot(436)
plt.hist(df[df['month']==6]['up'],bins=3)
plt.title('Jun')
plt.subplot(437)
plt.hist(df[df['month']==7]['up'],bins=3)
plt.title('Jul')
plt.subplot(438)
plt.hist(df[df['month']==8]['up'],bins=3)
plt.title('Aug')
plt.subplot(439)
plt.hist(df[df['month']==9]['up'],bins=3)
plt.title('Sep')
plt.subplot(4,3,10)
plt.hist(df[df['month']==10]['up'],bins=3)
plt.title('Oct')
plt.subplot(4,3,11)
plt.hist(df[df['month']==11]['up'],bins=3)
plt.title('Nov')
plt.subplot(4,3,12)
plt.hist(df[df['month']==12]['up'],bins=3)
plt.title('Dec')
plt.show()

# Split data
Here we are splitting the data between train and test. Since it does not look like a stratified data we are use a normal data split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, test_size = 0.25, random_state = RANDOM_SEED)

# Feature selection
We will use the PCA to evaluate the features and the sklearn.feature_selection.SelectKBest to evaluate the features considering the target values. The objective is to reduce it for a more reasonable number. In the moment we have 213 features. We will  reduce the number to components to 20.

In [ ]:
#pca = PCA()
#pca.fit(X_train,y_train)
#print pca.explained_variance_ratio_

#ica = FastICA(max_iter=20000, tol=0.1, random_state = RANDOM_SEED)
#ica.fit(X_train,y_train)

skb = SelectKBest(f_classif,k=30)
skb.fit(X_train,y_train)
#print skb.scores_

X_train_reduced = skb.transform(X_train)
X_test_reduced = skb.transform(X_test)

Note: First I started with PCA and k=20 in SelectKBest, as mentioned, but the results were not good. Trying to improve I included FastICA and increased the number of variables in the SelectKBest. It improved the performance.

# Defining the classifier
Since we want to know if the stock will go up in a given day or not it is a classification problem. Considering the amount of data that we have, over 2000, we will use the Support Vector Classifier.

For the classifier decision we are using the recommendation of the sklearn website, see image below:

![sklearn map](http://scikit-learn.org/stable/_static/ml_map.png)

Together with the classifier we will implement the GridSearchCV and the makescorer.

In [ ]:
svc = SVC(random_state = RANDOM_SEED)

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[1, 1.5, 3, 5, 10], 'degree':[2,3,4]}

precision_scorer = make_scorer(precision_score)

clf = GridSearchCV(svc, parameters, scoring = precision_scorer)

clf.fit(X_train_reduced, y_train)

print(clf.best_estimator_)
print(clf.best_score_)
svc = clf.best_estimator_

print("The score in the training data was %0.4f." % (precision_score(y_train,svc.predict(X_train_reduced))))
print("The score in the test data was %0.4f." % (precision_score(y_test,svc.predict(X_test_reduced))))


Since SVC is not working lets go to KNeighborsClassifier, as the map suggests.

In [ ]:
knc = KNeighborsClassifier()

parameters = {'n_neighbors':[2,4,5,8,16,32,64], 'p':[2, 3], 'leaf_size':[2,5,10,20,30]}

clf = GridSearchCV(knc, parameters, scoring = precision_scorer)

clf.fit(X_train_reduced, y_train)

print(clf.best_estimator_)
print(clf.best_score_)
knc = clf.best_estimator_

print("The score in the training data was %0.4f." % (precision_score(y_train,knc.predict(X_train_reduced))))
print("The score in the test data was %0.4f." % (precision_score(y_test,knc.predict(X_test_reduced))))

KNeighborsClassifier also did not work well. Let's try 2 ensemble methods, AdaBoostClassifier and RandomForestClassifier.

In [ ]:
abc = AdaBoostClassifier(random_state = RANDOM_SEED)

parameters = {'algorithm':('SAMME','SAMME.R'), 'n_estimators':[10,30,50,80,120,300,500,800,1200], 'learning_rate':[0.1,0.5,1.0,1.5,2.0]}

clf = GridSearchCV(abc, parameters, scoring = precision_scorer)

clf.fit(X_train_reduced, y_train)

print(clf.best_estimator_)
print(clf.best_score_)
abc = clf.best_estimator_

print("The score in the training data was %0.4f." % (precision_score(y_train,abc.predict(X_train_reduced))))
print("The score in the test data was %0.4f." % (precision_score(y_test,abc.predict(X_test_reduced))))

In [ ]:
rfc = RandomForestClassifier(n_jobs = -1, random_state = RANDOM_SEED)

#parameters = {'n_estimators':[10,30,50,80,120,300,500,800,1200], 'max_depth':[5,8,15,25,30,None],
#              'min_samples_split':[1,2,5,10,15,100], 'min_samples_leaf':[1,2,5,10], 'max_features':('sqrt','log2',None)}
parameters = {'n_estimators':[50,120,300,500,800,1200], 'max_depth':[5,15,30,None],
              'min_samples_split':[1,2,5,15,100], 'min_samples_leaf':[1,4,10], 'max_features':('sqrt','log2',None)}

clf = GridSearchCV(rfc, parameters, scoring = precision_scorer)

clf.fit(X_train_reduced, y_train)

print(clf.best_estimator_)
print(clf.best_score_)
rfc = clf.best_estimator_

print("The score in the training data was %0.4f." % (precision_score(y_train,rfc.predict(X_train_reduced))))
print("The score in the test data was %0.4f." % (precision_score(y_test,rfc.predict(X_test_reduced))))